In [2]:
import os
import sys
import time
import random
import warnings
import collections
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.experimental import enable_hist_gradient_boosting, enable_halving_search_cv  
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, HalvingRandomSearchCV 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor 

# from sklearn.linear_model import Ridge, Lasso, BayesianRidge, ElasticNet
from sklearn.preprocessing import OneHotEncoder

sys.path.append('../src')
import cb_utils
import cb_model_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

In [5]:
# configuration
use_cache = False
seed = random.randint(0, 100)

print(f'Seed: {seed}')

Seed: 19


In [6]:
query = f"select * from junk.ip_features_all;"
ip_features_all = cb_utils.sql_query_to_df(query, use_cache=use_cache)

Pulling query from db


In [7]:
ip_features_all.shape

(1444147, 140)

In [8]:
member_key = cb_utils.sql_query_to_df(f"select * from junk.ip_member_key;", use_cache=use_cache)

Pulling query from db


In [9]:
pre_months = 12
post_months = 12

In [7]:
# pre_post_months = pre_months + post_months

In [10]:
periods_df, months_df = cb_model_utils.build_member_periods(ip_features_all, pre_months=pre_months, post_months=post_months)

In [11]:
periods_df.head()

,is_cb_eligible,mco_id,mco_name,eom,member_id,line_of_business_id,ggroup,is_unaligned,age,gender,cwmm,days_in_month,cpmm,rx_tc,other_tc,ip_tc,er_tc,out_tc,snf_tc,icf_tc,hh_tc,amb_tc,hsp_tc,pro_tc,spc_fac_tc,dme_tc,cls_tc,hha_tc,hcbs_attdpcs_tc,hcbs_other_tc,hcbs_support_house_tc,hcbs_adult_day_tc,ip_ddos_span,er_ddos_span,out_ddos_span,snf_ddos_span,icf_ddos_span,hh_ddos_span,amb_ddos_span,hsp_ddos_span,pro_ddos_span,spc_fac_ddos_span,dme_ddos_span,cls_ddos_span,hha_ddos_span,hcbs_attdpcs_ddos_span,hcbs_other_ddos_span,hcbs_support_house_ddos_span,hcbs_adult_day_ddos_span,other_ddos_span,ip_ddos,er_ddos,out_ddos,snf_ddos,icf_ddos,hh_ddos,amb_ddos,hsp_ddos,pro_ddos,spc_fac_ddos,dme_ddos,cls_ddos,hha_ddos,hcbs_attdpcs_ddos,hcbs_other_ddos,hcbs_support_house_ddos,hcbs_adult_day_ddos,other_ddos,hcbs_pers_ddos,hcbs_assist_tech_ddos,oxygen_ddos,hosp_bed_ddos,chf_ddos,heart_ddos,copd_ddos,pulmonar_ddos,cancer_ddos,ckd_ddos,esrd_ddos,lipidy_ddos,diab_ddos,alzh_ddos,demented_ddos,stroke_ddos,hyper_ddos,fall_ddos,trans_ddos,liver_ddos,hippy_ddos,depressed_ddos,psycho_ddos,druggy_ddos,boozy_ddos,paralyzed_ddos,mono_ddos,mono_dom_ddos,hemi_ddos,hemi_dom_ddos,para_ddos,quad_ddos,tbi_ddos,obese_ddos,pressure_ulcer_ddos,hemophilia_ddos,hcbs_pers_tc,hcbs_assist_tech_tc,oxygen_tc,hosp_bed_tc,chf_tc,heart_tc,copd_tc,pulmonar_tc,cancer_tc,ckd_tc,esrd_tc,lipidy_tc,diab_tc,alzh_tc,demented_tc,stroke_tc,hyper_tc,fall_tc,trans_tc,liver_tc,hippy_tc,depressed_tc,psycho_tc,druggy_tc,boozy_tc,paralyzed_tc,mono_tc,mono_dom_tc,hemi_tc,hemi_dom_tc,para_tc,quad_tc,tbi_tc,obese_tc,pressure_ulcer_tc,hemophilia_tc,pre_0,pre_1,pre_2,pre_3,pre_4,pre_5,pre_6,pre_7,pre_8,pre_9,pre_10,pre_11,pre_12,pre_13,pre_14,pre_15,pre_16,pre_17,pre_18,pre_19,pre_20,pre_21,pre_22,pre_23,pre_24,pre_25,pre_26,pre_27,pre_28,pre_29,pre_30,pre_31,pre_32,pre_33,pre_34,pre_35,post_0,post_1,post_2,post_3,post_4,post_5,post_6,post_7,post_8,post_9,post_10,post_11,post_12,post_13,post_14,post_15,post_16,post_17,post_18,post_19,post_20,post_21,post_22,post_23,post_24,post_25,post_26,post_27,post_28,post_29,post_30,post_31,post_32,post_33,post_34,post_35,pre_post_elg_0,pre_post_elg_1,pre_post_elg_2,pre_post_elg_3,pre_post_elg_4,pre_post_elg_5,pre_post_elg_6,pre_post_elg_7,pre_post_elg_8,pre_post_elg_9,pre_post_elg_10,pre_post_elg_11,pre_post_elg_12,pre_post_elg_13,pre_post_elg_14,pre_post_elg_15,pre_post_elg_16,pre_post_elg_17,pre_post_elg_18,pre_post_elg_19,pre_post_elg_20,pre_post_elg_21,pre_post_elg_22,pre_post_elg_23,pre_post_elg_24,pre_post_elg_25,pre_post_elg_26,pre_post_elg_27,pre_post_elg_28,pre_post_elg_29,pre_post_elg_30,pre_post_elg_31,pre_post_elg_32,pre_post_elg_33,pre_post_elg_34,pre_post_elg_35
0,False,1,Anthem TN,2018-03-31,21009,1,0.0,False,54,m,1,31,1.0,34.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,274.51,None,38.49,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,None,1.0,None,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,1,Anthem TN,2018-04-30,21009,1,0.0,False,54,m,1,30,1.0,35.84,NaN,17014.8,1920.10,NaN,4200.0,NaN,NaN,610.15,NaN,5860.06,None,NaN,None,NaN,NaN,NaN,NaN,NaN,208.0,80.0,NaN,251.0,

In [11]:
months_df.head()

,eom
0,2017-01-31
1,2017-02-28
2,2017-03-31
3,2017-04-30
4,2017-05-31


In [12]:
# months_df.to_csv('./data/months_df.csv', index=False)

In [16]:
# # fill na's
# periods_df.is_cb_eligible = periods_df.is_cb_eligible.fillna(False)
# periods_df.is_unaligned = periods_df.is_unaligned.fillna(False)
# periods_df = periods_df.fillna(0)

# # add gender, state
# periods_df = periods_df.assign(is_male=np.where(periods_df.gender=='m',1,0))

# # assign state
# periods_df = periods_df.assign(state=periods_df.mco_name.str.split(' ').apply(lambda x: x[1]).replace({'Centene': 'IA'}))

### Build features + targets
Built separately with multithreaded script

### Train/Val/Test split
Avoid any leakage by doing the splits at the member level

In [13]:
# master_df = pd.read_parquet('./data/master_df.parquet')
# master_df = pd.read_parquet('./data/master_ddos_df.parquet')
master_df = pd.read_parquet('./data/final_wide_df_v9_all.parquet')
# master_df = master_df.loc[master_df.period > 24]
# make dtype str for these categorical features
# master_df.ggroup = master_df.ggroup.astype(str)
# master_df.line_of_business_id = master_df.line_of_business_id.astype(str)

In [14]:
master_df.head()

,ip_ddos_span_0,er_ddos_span_0,out_ddos_span_0,snf_ddos_span_0,icf_ddos_span_0,hh_ddos_span_0,amb_ddos_span_0,hsp_ddos_span_0,pro_ddos_span_0,spc_fac_ddos_span_0,dme_ddos_span_0,cls_ddos_span_0,hha_ddos_span_0,hcbs_attdpcs_ddos_span_0,hcbs_other_ddos_span_0,hcbs_support_house_ddos_span_0,hcbs_adult_day_ddos_span_0,other_ddos_span_0,ip_ddos_0,er_ddos_0,out_ddos_0,snf_ddos_0,icf_ddos_0,hh_ddos_0,amb_ddos_0,hsp_ddos_0,pro_ddos_0,spc_fac_ddos_0,dme_ddos_0,cls_ddos_0,hha_ddos_0,hcbs_attdpcs_ddos_0,hcbs_other_ddos_0,hcbs_support_house_ddos_0,hcbs_adult_day_ddos_0,other_ddos_0,hcbs_pers_ddos_0,hcbs_assist_tech_ddos_0,oxygen_ddos_0,hosp_bed_ddos_0,chf_ddos_0,heart_ddos_0,copd_ddos_0,pulmonar_ddos_0,cancer_ddos_0,ckd_ddos_0,esrd_ddos_0,lipidy_ddos_0,diab_ddos_0,alzh_ddos_0,demented_ddos_0,stroke_ddos_0,hyper_ddos_0,fall_ddos_0,trans_ddos_0,liver_ddos_0,hippy_ddos_0,depressed_ddos_0,psycho_ddos_0,druggy_ddos_0,boozy_ddos_0,paralyzed_ddos_0,mono_ddos_0,mono_dom_ddos_0,hemi_ddos_0,hemi_dom_ddos_0,para_ddos_0,quad_ddos_0,tbi_ddos_0,obese_ddos_0,pressure_ulcer_ddos_0,hemophilia_ddos_0,rx_tc_0,other_tc_0,ip_tc_0,er_tc_0,out_tc_0,snf_tc_0,icf_tc_0,hh_tc_0,amb_tc_0,hsp_tc_0,pro_tc_0,spc_fac_tc_0,dme_tc_0,cls_tc_0,hha_tc_0,hcbs_attdpcs_tc_0,hcbs_other_tc_0,hcbs_support_house_tc_0,hcbs_adult_day_tc_0,hcbs_pers_tc_0,hcbs_assist_tech_tc_0,oxygen_tc_0,hosp_bed_tc_0,chf_tc_0,heart_tc_0,copd_tc_0,pulmonar_tc_0,cancer_tc_0,ckd_tc_0,esrd_tc_0,lipidy_tc_0,diab_tc_0,alzh_tc_0,demented_tc_0,stroke_tc_0,hyper_tc_0,fall_tc_0,trans_tc_0,liver_tc_0,hippy_tc_0,depressed_tc_0,psycho_tc_0,druggy_tc_0,boozy_tc_0,paralyzed_tc_0,mono_tc_0,mono_dom_tc_0,hemi_tc_0,hemi_dom_tc_0,para_tc_0,quad_tc_0,tbi_tc_0,obese_tc_0,pressure_ulcer_tc_0,hemophilia_tc_0,ip_ddos_span_1,er_ddos_span_1,out_ddos_span_1,snf_ddos_span_1,icf_ddos_span_1,hh_ddos_span_1,amb_ddos_span_1,hsp_ddos_span_1,pro_ddos_span_1,spc_fac_ddos_span_1,dme_ddos_span_1,cls_ddos_span_1,hha_ddos_span_1,hcbs_attdpcs_ddos_span_1,hcbs_other_ddos_span_1,hcbs_support_house_ddos_span_1,hcbs_adult_day_ddos_span_1,other_ddos_span_1,ip_ddos_1,er_ddos_1,out_ddos_1,snf_ddos_1,icf_ddos_1,hh_ddos_1,amb_ddos_1,hsp_ddos_1,pro_ddos_1,spc_fac_ddos_1,dme_ddos_1,cls_ddos_1,hha_ddos_1,hcbs_attdpcs_ddos_1,hcbs_other_ddos_1,hcbs_support_house_ddos_1,hcbs_adult_day_ddos_1,other_ddos_1,hcbs_pers_ddos_1,hcbs_assist_tech_ddos_1,oxygen_ddos_1,hosp_bed_ddos_1,chf_ddos_1,heart_ddos_1,copd_ddos_1,pulmonar_ddos_1,cancer_ddos_1,ckd_ddos_1,esrd_ddos_1,lipidy_ddos_1,diab_ddos_1,alzh_ddos_1,demented_ddos_1,stroke_ddos_1,hyper_ddos_1,fall_ddos_1,trans_ddos_1,liver_ddos_1,hippy_ddos_1,depressed_ddos_1,psycho_ddos_1,druggy_ddos_1,boozy_ddos_1,paralyzed_ddos_1,mono_ddos_1,mono_dom_ddos_1,hemi_ddos_1,hemi_dom_ddos_1,para_ddos_1,quad_ddos_1,tbi_ddos_1,obese_ddos_1,pressure_ulcer_ddos_1,hemophilia_ddos_1,rx_tc_1,other_tc_1,ip_tc_1,er_tc_1,out_tc_1,snf_tc_1,icf_tc_1,hh_tc_1,amb_tc_1,hsp_tc_1,pro_tc_1,spc_fac_tc_1,dme_tc_1,cls_tc_1,hha_tc_1,hcbs_attdpcs_tc_1,hcbs_other_tc_1,hcbs_support_house_tc_1,hcbs_adult_day_tc_1,hcbs_pers_tc_1,hcbs_assist_tech_tc_1,oxygen_tc_1,hosp_bed_tc_1,chf_tc_1,heart_tc_1,copd_tc_1,pulmonar_tc_1,cancer_tc_1,ckd_tc_1,esrd_tc_1,lipidy_tc_1,diab_tc_1,alzh_tc_1,demented_tc_1,stroke_tc_1,hyper_tc_1,fall_tc_1,trans_tc_1,liver_tc_1,hippy_tc_1,depressed_tc_1,psycho_tc_1,druggy_tc_1,boozy_tc_1,paralyzed_tc_1,mono_tc_1,mono_dom_tc_1,hemi_tc_1,hemi_dom_tc_1,para_tc_1,quad_tc_1,...,hosp_bed_ddos_10,chf_ddos_10,heart_ddos_10,copd_ddos_10,pulmonar_ddos_10,cancer_ddos_10,ckd_ddos_10,esrd_ddos_10,lipidy_ddos_10,diab_ddos_10,alzh_ddos_10,demented_ddos_10,stroke_ddos_10,hyper_ddos_10,fall_ddos_10,trans_ddos_10,liver_ddos_10,hippy_ddos_10,depressed_ddos_10,psycho_ddos_10,druggy_ddos_10,boozy_ddos_10,paralyzed_ddos_10,mono_ddos_10,mono_dom_ddos_10,hemi_ddos_10,hemi_dom_ddos_10,para_ddos_10,quad_ddos_10,tbi_ddos_10,obese_ddos_10,pressure_ulcer_ddos_10,hemophilia_ddos_10,rx_tc_10,other_tc_10,ip_tc_10,er_tc_10,out_tc_10,snf_tc_10,icf_tc_10,hh_tc_10,am

### Build Yearly DDOS service_types + Demographics

In [22]:
d = cb_model_utils.build_yearly_stddos_dem(master_df)
# train_val_test_split(d, 'yearly_st_ddos_dem', return_wo_saving=True)
train, val, test =  cb_model_utils.train_val_test_split(d, return_wo_saving=True)

train.head()

In [39]:
perf

{'train_score': 0.20769689879588693,
 'val_score': 0.18612603153270502,
 'train_mae': 10756.070996174036,
 'val_mae': 10746.279899105955}

In [24]:
x_train, y_train = cb_model_utils.get_xy(train)

In [29]:
x_train.head()

,ip_ddos,er_ddos,out_ddos,snf_ddos,icf_ddos,hh_ddos,amb_ddos,hsp_ddos,pro_ddos,spc_fac_ddos,dme_ddos,cls_ddos,hha_ddos,is_state_TN,is_state_FL,is_state_TX,is_state_KS,is_state_IA,is_lob_1,is_lob_3,is_lob_2,is_lob_8,is_group_0.0,is_group_3.0,is_group_2.0,is_group_1.0,is_group_-1.0,is_group_5.0,is_group_6.0,is_group_4.0,is_group_8.0,is_group_14.0,is_group_16.0,is_group_7.0,is_group_11.0,is_group_15.0,is_group_20.0,is_group_12.0,is_group_13.0,is_group_9.0,is_group_21.0,is_group_18.0,is_group_10.0,is_male,age
0,0.0,0.0,8.0,0.0,0.0,39.0,0.0,0.0,11.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0
1,0.0,11.0,155.0,0.0,0.0,0.0,52.0,0.0,54.0,0.0,47.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0
2,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.0
3,0.0,0.0,2.0,0.0,0.0,49.0,0.0,0.0,33.0,0.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0
4,0.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,21.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64.0


In [25]:
histr = HistGradientBoostingRegressor()
histr.fit(x_train, y_train)


HistGradientBoostingRegressor()

In [27]:
perf = cb_model_utils.get_model_performance(histr, train, val)

In [28]:
perf

{'train_score': 0.27172380699798515,
 'val_score': 0.16912553017467025,
 'train_mae': 10581.072524029842,
 'val_mae': 10812.120463940097}

In [37]:
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV 

param_grid = {
    'l2_regularization': [.5, 1, 2],
    'learning_rate': [0.01, 0.1, .2, .5, .7, 1],
    'min_samples_leaf': [20, 50, 100, 200],
    'max_iter': [100, 200, 500],
    'max_depth': [3, 5, 10, 20, 50],
}


base_estimator = HistGradientBoostingRegressor()
sh = HalvingRandomSearchCV(base_estimator,
                         param_grid,
                         cv=5,
                         factor=2,
                         n_candidates='exhaust',
                         min_resources=5000,
                         scoring='neg_mean_squared_error',
                         n_jobs=-1
                        ).fit(x_train, y_train)
sh.n_resources_

[5000, 10000, 20000, 40000, 80000, 160000, 320000]

In [40]:
sh.best_params_

{'min_samples_leaf': 50,
 'max_iter': 100,
 'max_depth': 5,
 'learning_rate': 0.1,
 'l2_regularization': 2}

In [38]:
perf = cb_model_utils.get_model_performance(sh.best_estimator_, train, val)

In [36]:
perf

{'train_score': 0.2701421790508892,
 'val_score': 0.16328924263016642,
 'train_mae': 10447.03850319733,
 'val_mae': 10828.615643337787}

In [39]:
perf

{'train_score': 0.20769689879588693,
 'val_score': 0.18612603153270502,
 'train_mae': 10756.070996174036,
 'val_mae': 10746.279899105955}

### Build MoM DDOS service_types + Demographics

In [67]:
d = cb_model_utils.build_mom_stddos_dem(master_df)
# train_val_test_split(d, 'yearly_st_ddos_dem', return_wo_saving=True)
train, val, test =  cb_model_utils.train_val_test_split(d, return_wo_saving=True)

train.head()

,ip_ddos_0,ip_ddos_1,ip_ddos_2,ip_ddos_3,ip_ddos_4,ip_ddos_5,ip_ddos_6,ip_ddos_7,ip_ddos_8,ip_ddos_9,ip_ddos_10,ip_ddos_11,er_ddos_0,er_ddos_1,er_ddos_2,er_ddos_3,er_ddos_4,er_ddos_5,er_ddos_6,er_ddos_7,er_ddos_8,er_ddos_9,er_ddos_10,er_ddos_11,out_ddos_0,out_ddos_1,out_ddos_2,out_ddos_3,out_ddos_4,out_ddos_5,out_ddos_6,out_ddos_7,out_ddos_8,out_ddos_9,out_ddos_10,out_ddos_11,snf_ddos_0,snf_ddos_1,snf_ddos_2,snf_ddos_3,snf_ddos_4,snf_ddos_5,snf_ddos_6,snf_ddos_7,snf_ddos_8,snf_ddos_9,snf_ddos_10,snf_ddos_11,icf_ddos_0,icf_ddos_1,icf_ddos_2,icf_ddos_3,icf_ddos_4,icf_ddos_5,icf_ddos_6,icf_ddos_7,icf_ddos_8,icf_ddos_9,icf_ddos_10,icf_ddos_11,hh_ddos_0,hh_ddos_1,hh_ddos_2,hh_ddos_3,hh_ddos_4,hh_ddos_5,hh_ddos_6,hh_ddos_7,hh_ddos_8,hh_ddos_9,hh_ddos_10,hh_ddos_11,amb_ddos_0,amb_ddos_1,amb_ddos_2,amb_ddos_3,amb_ddos_4,amb_ddos_5,amb_ddos_6,amb_ddos_7,amb_ddos_8,amb_ddos_9,amb_ddos_10,amb_ddos_11,hsp_ddos_0,hsp_ddos_1,hsp_ddos_2,hsp_ddos_3,hsp_ddos_4,hsp_ddos_5,hsp_ddos_6,hsp_ddos_7,hsp_ddos_8,hsp_ddos_9,hsp_ddos_10,hsp_ddos_11,pro_ddos_0,pro_ddos_1,pro_ddos_2,pro_ddos_3,pro_ddos_4,pro_ddos_5,pro_ddos_6,pro_ddos_7,pro_ddos_8,pro_ddos_9,pro_ddos_10,pro_ddos_11,spc_fac_ddos_0,spc_fac_ddos_1,spc_fac_ddos_2,spc_fac_ddos_3,spc_fac_ddos_4,spc_fac_ddos_5,spc_fac_ddos_6,spc_fac_ddos_7,spc_fac_ddos_8,spc_fac_ddos_9,spc_fac_ddos_10,spc_fac_ddos_11,dme_ddos_0,dme_ddos_1,dme_ddos_2,dme_ddos_3,dme_ddos_4,dme_ddos_5,dme_ddos_6,dme_ddos_7,dme_ddos_8,dme_ddos_9,dme_ddos_10,dme_ddos_11,cls_ddos_0,cls_ddos_1,cls_ddos_2,cls_ddos_3,cls_ddos_4,cls_ddos_5,cls_ddos_6,cls_ddos_7,cls_ddos_8,cls_ddos_9,cls_ddos_10,cls_ddos_11,hha_ddos_0,hha_ddos_1,hha_ddos_2,hha_ddos_3,hha_ddos_4,hha_ddos_5,hha_ddos_6,hha_ddos_7,hha_ddos_8,hha_ddos_9,hha_ddos_10,hha_ddos_11,is_state_TN,is_state_FL,is_state_TX,is_state_KS,is_state_IA,is_lob_1,is_lob_3,is_lob_2,is_lob_8,is_group_0.0,is_group_3.0,is_group_2.0,is_group_1.0,is_group_-1.0,is_group_5.0,is_group_6.0,is_group_4.0,is_group_8.0,is_group_14.0,is_group_16.0,is_group_7.0,is_group_11.0,is_group_15.0,is_group_20.0,is_group_12.0,is_group_13.0,is_group_9.0,is_group_21.0,is_group_18.0,is_group_10.0,is_male,age,member_id,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,4.0,0.0,0.0,0.0,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0,26.0,0.00
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,2.0,13.0,15.0,13.0,15.0,16.0,18.0,15.0,15.0,16.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,7.0,9.0,13.0,4.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,8.0,3.0,10.0,3.0,3.0,5.0,5.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,12.0,7.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,102.0,58823.36
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,7.0,2.0,6.0,9.0,8.

In [72]:
list(train.columns)

['ip_ddos_0',
 'ip_ddos_1',
 'ip_ddos_2',
 'ip_ddos_3',
 'ip_ddos_4',
 'ip_ddos_5',
 'ip_ddos_6',
 'ip_ddos_7',
 'ip_ddos_8',
 'ip_ddos_9',
 'ip_ddos_10',
 'ip_ddos_11',
 'er_ddos_0',
 'er_ddos_1',
 'er_ddos_2',
 'er_ddos_3',
 'er_ddos_4',
 'er_ddos_5',
 'er_ddos_6',
 'er_ddos_7',
 'er_ddos_8',
 'er_ddos_9',
 'er_ddos_10',
 'er_ddos_11',
 'out_ddos_0',
 'out_ddos_1',
 'out_ddos_2',
 'out_ddos_3',
 'out_ddos_4',
 'out_ddos_5',
 'out_ddos_6',
 'out_ddos_7',
 'out_ddos_8',
 'out_ddos_9',
 'out_ddos_10',
 'out_ddos_11',
 'snf_ddos_0',
 'snf_ddos_1',
 'snf_ddos_2',
 'snf_ddos_3',
 'snf_ddos_4',
 'snf_ddos_5',
 'snf_ddos_6',
 'snf_ddos_7',
 'snf_ddos_8',
 'snf_ddos_9',
 'snf_ddos_10',
 'snf_ddos_11',
 'icf_ddos_0',
 'icf_ddos_1',
 'icf_ddos_2',
 'icf_ddos_3',
 'icf_ddos_4',
 'icf_ddos_5',
 'icf_ddos_6',
 'icf_ddos_7',
 'icf_ddos_8',
 'icf_ddos_9',
 'icf_ddos_10',
 'icf_ddos_11',
 'hh_ddos_0',
 'hh_ddos_1',
 'hh_ddos_2',
 'hh_ddos_3',
 'hh_ddos_4',
 'hh_ddos_5',
 'hh_ddos_6',
 'hh_ddos_7',
 '

In [43]:
x_train, y_train = cb_model_utils.get_xy(train)

In [44]:
x_train.head()

,ip_ddos_0,ip_ddos_1,ip_ddos_2,ip_ddos_3,ip_ddos_4,ip_ddos_5,ip_ddos_6,ip_ddos_7,ip_ddos_8,ip_ddos_9,ip_ddos_10,ip_ddos_11,er_ddos_0,er_ddos_1,er_ddos_2,er_ddos_3,er_ddos_4,er_ddos_5,er_ddos_6,er_ddos_7,er_ddos_8,er_ddos_9,er_ddos_10,er_ddos_11,out_ddos_0,out_ddos_1,out_ddos_2,out_ddos_3,out_ddos_4,out_ddos_5,out_ddos_6,out_ddos_7,out_ddos_8,out_ddos_9,out_ddos_10,out_ddos_11,snf_ddos_0,snf_ddos_1,snf_ddos_2,snf_ddos_3,snf_ddos_4,snf_ddos_5,snf_ddos_6,snf_ddos_7,snf_ddos_8,snf_ddos_9,snf_ddos_10,snf_ddos_11,icf_ddos_0,icf_ddos_1,icf_ddos_2,icf_ddos_3,icf_ddos_4,icf_ddos_5,icf_ddos_6,icf_ddos_7,icf_ddos_8,icf_ddos_9,icf_ddos_10,icf_ddos_11,hh_ddos_0,hh_ddos_1,hh_ddos_2,hh_ddos_3,hh_ddos_4,hh_ddos_5,hh_ddos_6,hh_ddos_7,hh_ddos_8,hh_ddos_9,hh_ddos_10,hh_ddos_11,amb_ddos_0,amb_ddos_1,amb_ddos_2,amb_ddos_3,amb_ddos_4,amb_ddos_5,amb_ddos_6,amb_ddos_7,amb_ddos_8,amb_ddos_9,amb_ddos_10,amb_ddos_11,hsp_ddos_0,hsp_ddos_1,hsp_ddos_2,hsp_ddos_3,hsp_ddos_4,hsp_ddos_5,hsp_ddos_6,hsp_ddos_7,hsp_ddos_8,hsp_ddos_9,hsp_ddos_10,hsp_ddos_11,pro_ddos_0,pro_ddos_1,pro_ddos_2,pro_ddos_3,pro_ddos_4,pro_ddos_5,pro_ddos_6,pro_ddos_7,pro_ddos_8,pro_ddos_9,pro_ddos_10,pro_ddos_11,spc_fac_ddos_0,spc_fac_ddos_1,spc_fac_ddos_2,spc_fac_ddos_3,spc_fac_ddos_4,spc_fac_ddos_5,spc_fac_ddos_6,spc_fac_ddos_7,spc_fac_ddos_8,spc_fac_ddos_9,spc_fac_ddos_10,spc_fac_ddos_11,dme_ddos_0,dme_ddos_1,dme_ddos_2,dme_ddos_3,dme_ddos_4,dme_ddos_5,dme_ddos_6,dme_ddos_7,dme_ddos_8,dme_ddos_9,dme_ddos_10,dme_ddos_11,cls_ddos_0,cls_ddos_1,cls_ddos_2,cls_ddos_3,cls_ddos_4,cls_ddos_5,cls_ddos_6,cls_ddos_7,cls_ddos_8,cls_ddos_9,cls_ddos_10,cls_ddos_11,hha_ddos_0,hha_ddos_1,hha_ddos_2,hha_ddos_3,hha_ddos_4,hha_ddos_5,hha_ddos_6,hha_ddos_7,hha_ddos_8,hha_ddos_9,hha_ddos_10,hha_ddos_11,is_state_TN,is_state_FL,is_state_TX,is_state_KS,is_state_IA,is_lob_1,is_lob_3,is_lob_2,is_lob_8,is_group_0.0,is_group_3.0,is_group_2.0,is_group_1.0,is_group_-1.0,is_group_5.0,is_group_6.0,is_group_4.0,is_group_8.0,is_group_14.0,is_group_16.0,is_group_7.0,is_group_11.0,is_group_15.0,is_group_20.0,is_group_12.0,is_group_13.0,is_group_9.0,is_group_21.0,is_group_18.0,is_group_10.0,is_male,age
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,4.0,0.0,0.0,0.0,10.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,2.0,13.0,15.0,13.0,15.0,16.0,18.0,15.0,15.0,16.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,7.0,9.0,13.0,4.0,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,8.0,3.0,10.0,3.0,3.0,5.0,5.0,4.0,5.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,12.0,7.0,1.0,1.0,3.0,1.0,2.0,2.0,2.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [45]:
histr = HistGradientBoostingRegressor()
histr.fit(x_train, y_train)

HistGradientBoostingRegressor()

In [46]:
perf = cb_model_utils.get_model_performance(histr, train, val)

In [47]:
perf

{'train_score': 0.2454720202509305,
 'val_score': 0.17870177931878617,
 'train_mae': 10420.3411134222,
 'val_mae': 10750.360952066854}

In [48]:

param_grid = {
    'l2_regularization': [.5, 1, 2],
    'learning_rate': [0.01, 0.1, .2, .5, .7, 1],
    'min_samples_leaf': [20, 50, 100, 200],
    'max_iter': [100, 200, 500],
    'max_depth': [3, 5, 10, 20, 50],
}


base_estimator = HistGradientBoostingRegressor()
sh = HalvingRandomSearchCV(base_estimator,
                         param_grid,
                         cv=5,
                         factor=2,
                         n_candidates='exhaust',
                         min_resources=5000,
                         scoring='neg_mean_squared_error',
                         n_jobs=-1
                        ).fit(x_train, y_train)
sh.n_resources_

/Users/bp/miniforge3/envs/data-analytics/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[5000, 10000, 20000, 40000, 80000, 160000, 320000]

In [49]:
sh.best_params_

{'min_samples_leaf': 100,
 'max_iter': 100,
 'max_depth': 10,
 'learning_rate': 0.1,
 'l2_regularization': 0.5}

In [50]:
perf = cb_model_utils.get_model_performance(sh.best_estimator_, train, val)

In [51]:
perf

{'train_score': 0.22128204591848655,
 'val_score': 0.1814543658331561,
 'train_mae': 10453.3834366035,
 'val_mae': 10735.850339099828}

### Yearly st DDOS + dx ddos  + Dem

In [52]:
d = cb_model_utils.build_yearly_stdxddos_dem(master_df)
# train_val_test_split(d, 'yearly_st_ddos_dem', return_wo_saving=True)
train, val, test =  cb_model_utils.train_val_test_split(d, return_wo_saving=True)

train.head()

,oxygen_ddos,hosp_bed_ddos,chf_ddos,heart_ddos,copd_ddos,pulmonar_ddos,cancer_ddos,ckd_ddos,esrd_ddos,lipidy_ddos,diab_ddos,alzh_ddos,demented_ddos,stroke_ddos,hyper_ddos,fall_ddos,trans_ddos,liver_ddos,hippy_ddos,depressed_ddos,psycho_ddos,druggy_ddos,boozy_ddos,paralyzed_ddos,mono_ddos,mono_dom_ddos,hemi_ddos,hemi_dom_ddos,para_ddos,quad_ddos,tbi_ddos,obese_ddos,pressure_ulcer_ddos,hemophilia_ddos,ip_ddos,er_ddos,out_ddos,snf_ddos,icf_ddos,hh_ddos,amb_ddos,hsp_ddos,pro_ddos,spc_fac_ddos,dme_ddos,cls_ddos,hha_ddos,is_state_TN,is_state_FL,is_state_TX,is_state_KS,is_state_IA,is_lob_1,is_lob_3,is_lob_2,is_lob_8,is_group_0.0,is_group_3.0,is_group_2.0,is_group_1.0,is_group_-1.0,is_group_5.0,is_group_6.0,is_group_4.0,is_group_8.0,is_group_14.0,is_group_16.0,is_group_7.0,is_group_11.0,is_group_15.0,is_group_20.0,is_group_12.0,is_group_13.0,is_group_9.0,is_group_21.0,is_group_18.0,is_group_10.0,is_male,age,member_id,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,8.0,0.0,0.0,39.0,0.0,0.0,11.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0,26.0,0.00
1,0.0,0.0,2.0,3.0,14.0,0.0,0.0,169.0,9.0,1.0,40.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,11.0,155.0,0.0,0.0,0.0,52.0,0.0,54.0,0.0,47.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,102.0,58823.36
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,49.0,0.0,0.0,33.0,0.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,185.0,5860.94
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,21.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64.0,200.0,0.00
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0,212.0,78206.40


In [53]:
x_train, y_train = cb_model_utils.get_xy(train)

In [54]:
x_train.head()

,oxygen_ddos,hosp_bed_ddos,chf_ddos,heart_ddos,copd_ddos,pulmonar_ddos,cancer_ddos,ckd_ddos,esrd_ddos,lipidy_ddos,diab_ddos,alzh_ddos,demented_ddos,stroke_ddos,hyper_ddos,fall_ddos,trans_ddos,liver_ddos,hippy_ddos,depressed_ddos,psycho_ddos,druggy_ddos,boozy_ddos,paralyzed_ddos,mono_ddos,mono_dom_ddos,hemi_ddos,hemi_dom_ddos,para_ddos,quad_ddos,tbi_ddos,obese_ddos,pressure_ulcer_ddos,hemophilia_ddos,ip_ddos,er_ddos,out_ddos,snf_ddos,icf_ddos,hh_ddos,amb_ddos,hsp_ddos,pro_ddos,spc_fac_ddos,dme_ddos,cls_ddos,hha_ddos,is_state_TN,is_state_FL,is_state_TX,is_state_KS,is_state_IA,is_lob_1,is_lob_3,is_lob_2,is_lob_8,is_group_0.0,is_group_3.0,is_group_2.0,is_group_1.0,is_group_-1.0,is_group_5.0,is_group_6.0,is_group_4.0,is_group_8.0,is_group_14.0,is_group_16.0,is_group_7.0,is_group_11.0,is_group_15.0,is_group_20.0,is_group_12.0,is_group_13.0,is_group_9.0,is_group_21.0,is_group_18.0,is_group_10.0,is_male,age
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,8.0,0.0,0.0,39.0,0.0,0.0,11.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,35.0
1,0.0,0.0,2.0,3.0,14.0,0.0,0.0,169.0,9.0,1.0,40.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,11.0,155.0,0.0,0.0,0.0,52.0,0.0,54.0,0.0,47.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0
3,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,49.0,0.0,0.0,33.0,0.0,9.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,10.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,21.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,64.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53.0


In [55]:
histr = HistGradientBoostingRegressor()
histr.fit(x_train, y_train)

HistGradientBoostingRegressor()

In [57]:
perf = cb_model_utils.get_model_performance(histr, train, val)
perf

{'train_score': 0.3215135948612582,
 'val_score': 0.16742935002095427,
 'train_mae': 10231.275542577412,
 'val_mae': 10979.126708163685}

In [58]:

param_grid = {
    'l2_regularization': [.5, 1, 2],
    'learning_rate': [0.01, 0.1, .2, .5, .7, 1],
    'min_samples_leaf': [20, 50, 100, 200],
    'max_iter': [100, 200, 500],
    'max_depth': [3, 5, 10, 20, 50],
}


base_estimator = HistGradientBoostingRegressor()
sh = HalvingRandomSearchCV(base_estimator,
                         param_grid,
                         cv=5,
                         factor=2,
                         n_candidates='exhaust',
                         min_resources=5000,
                         scoring='neg_mean_squared_error',
                         n_jobs=-1
                        ).fit(x_train, y_train)
sh.best_params_

{'min_samples_leaf': 20,
 'max_iter': 100,
 'max_depth': 50,
 'learning_rate': 0.1,
 'l2_regularization': 0.5}

In [59]:
perf = cb_model_utils.get_model_performance(sh.best_estimator_, train, val)
perf

{'train_score': 0.32356710961736446,
 'val_score': 0.15941442497608638,
 'train_mae': 10243.069163723785,
 'val_mae': 10993.645446332544}

### MOM st DDOS + dx ddos  + Dem

In [60]:
d = cb_model_utils.build_mom_stdxddos_dem(master_df)
# train_val_test_split(d, 'yearly_st_ddos_dem', return_wo_saving=True)
train, val, test =  cb_model_utils.train_val_test_split(d, return_wo_saving=True)

train.head()

,oxygen_ddos_0,oxygen_ddos_1,oxygen_ddos_2,oxygen_ddos_3,oxygen_ddos_4,oxygen_ddos_5,oxygen_ddos_6,oxygen_ddos_7,oxygen_ddos_8,oxygen_ddos_9,oxygen_ddos_10,oxygen_ddos_11,hosp_bed_ddos_0,hosp_bed_ddos_1,hosp_bed_ddos_2,hosp_bed_ddos_3,hosp_bed_ddos_4,hosp_bed_ddos_5,hosp_bed_ddos_6,hosp_bed_ddos_7,hosp_bed_ddos_8,hosp_bed_ddos_9,hosp_bed_ddos_10,hosp_bed_ddos_11,chf_ddos_0,chf_ddos_1,chf_ddos_2,chf_ddos_3,chf_ddos_4,chf_ddos_5,chf_ddos_6,chf_ddos_7,chf_ddos_8,chf_ddos_9,chf_ddos_10,chf_ddos_11,heart_ddos_0,heart_ddos_1,heart_ddos_2,heart_ddos_3,heart_ddos_4,heart_ddos_5,heart_ddos_6,heart_ddos_7,heart_ddos_8,heart_ddos_9,heart_ddos_10,heart_ddos_11,copd_ddos_0,copd_ddos_1,copd_ddos_2,copd_ddos_3,copd_ddos_4,copd_ddos_5,copd_ddos_6,copd_ddos_7,copd_ddos_8,copd_ddos_9,copd_ddos_10,copd_ddos_11,pulmonar_ddos_0,pulmonar_ddos_1,pulmonar_ddos_2,pulmonar_ddos_3,pulmonar_ddos_4,pulmonar_ddos_5,pulmonar_ddos_6,pulmonar_ddos_7,pulmonar_ddos_8,pulmonar_ddos_9,pulmonar_ddos_10,pulmonar_ddos_11,cancer_ddos_0,cancer_ddos_1,cancer_ddos_2,cancer_ddos_3,cancer_ddos_4,cancer_ddos_5,cancer_ddos_6,cancer_ddos_7,cancer_ddos_8,cancer_ddos_9,cancer_ddos_10,cancer_ddos_11,ckd_ddos_0,ckd_ddos_1,ckd_ddos_2,ckd_ddos_3,ckd_ddos_4,ckd_ddos_5,ckd_ddos_6,ckd_ddos_7,ckd_ddos_8,ckd_ddos_9,ckd_ddos_10,ckd_ddos_11,esrd_ddos_0,esrd_ddos_1,esrd_ddos_2,esrd_ddos_3,esrd_ddos_4,esrd_ddos_5,esrd_ddos_6,esrd_ddos_7,esrd_ddos_8,esrd_ddos_9,esrd_ddos_10,esrd_ddos_11,lipidy_ddos_0,lipidy_ddos_1,lipidy_ddos_2,lipidy_ddos_3,lipidy_ddos_4,lipidy_ddos_5,lipidy_ddos_6,lipidy_ddos_7,lipidy_ddos_8,lipidy_ddos_9,lipidy_ddos_10,lipidy_ddos_11,diab_ddos_0,diab_ddos_1,diab_ddos_2,diab_ddos_3,diab_ddos_4,diab_ddos_5,diab_ddos_6,diab_ddos_7,diab_ddos_8,diab_ddos_9,diab_ddos_10,diab_ddos_11,alzh_ddos_0,alzh_ddos_1,alzh_ddos_2,alzh_ddos_3,alzh_ddos_4,alzh_ddos_5,alzh_ddos_6,alzh_ddos_7,alzh_ddos_8,alzh_ddos_9,alzh_ddos_10,alzh_ddos_11,demented_ddos_0,demented_ddos_1,demented_ddos_2,demented_ddos_3,demented_ddos_4,demented_ddos_5,demented_ddos_6,demented_ddos_7,demented_ddos_8,demented_ddos_9,demented_ddos_10,demented_ddos_11,stroke_ddos_0,stroke_ddos_1,stroke_ddos_2,stroke_ddos_3,stroke_ddos_4,stroke_ddos_5,stroke_ddos_6,stroke_ddos_7,stroke_ddos_8,stroke_ddos_9,stroke_ddos_10,stroke_ddos_11,hyper_ddos_0,hyper_ddos_1,hyper_ddos_2,hyper_ddos_3,hyper_ddos_4,hyper_ddos_5,hyper_ddos_6,hyper_ddos_7,hyper_ddos_8,hyper_ddos_9,hyper_ddos_10,hyper_ddos_11,fall_ddos_0,fall_ddos_1,fall_ddos_2,fall_ddos_3,fall_ddos_4,fall_ddos_5,fall_ddos_6,fall_ddos_7,fall_ddos_8,fall_ddos_9,fall_ddos_10,fall_ddos_11,trans_ddos_0,trans_ddos_1,trans_ddos_2,trans_ddos_3,trans_ddos_4,trans_ddos_5,trans_ddos_6,trans_ddos_7,trans_ddos_8,trans_ddos_9,trans_ddos_10,trans_ddos_11,liver_ddos_0,liver_ddos_1,liver_ddos_2,liver_ddos_3,liver_ddos_4,liver_ddos_5,liver_ddos_6,liver_ddos_7,liver_ddos_8,liver_ddos_9,liver_ddos_10,liver_ddos_11,hippy_ddos_0,hippy_ddos_1,hippy_ddos_2,hippy_ddos_3,hippy_ddos_4,hippy_ddos_5,hippy_ddos_6,hippy_ddos_7,hippy_ddos_8,hippy_ddos_9,hippy_ddos_10,hippy_ddos_11,depressed_ddos_0,depressed_ddos_1,depressed_ddos_2,depressed_ddos_3,depressed_ddos_4,depressed_ddos_5,depressed_ddos_6,depressed_ddos_7,depressed_ddos_8,depressed_ddos_9,depressed_ddos_10,depressed_ddos_11,psycho_ddos_0,psycho_ddos_1,psycho_ddos_2,psycho_ddos_3,psycho_ddos_4,psycho_ddos_5,psycho_ddos_6,psycho_ddos_7,psycho_ddos_8,psycho_ddos_9,...,quad_ddos_0,quad_ddos_1,quad_ddos_2,quad_ddos_3,quad_ddos_4,quad_ddos_5,quad_ddos_6,quad_ddos_7,quad_ddos_8,quad_ddos_9,quad_ddos_10,quad_ddos_11,tbi_ddos_0,tbi_ddos_1,tbi_ddos_2,tbi_ddos_3,tbi_ddos_4,tbi_ddos_5,tbi_ddos_6,tbi_ddos_7,tbi_ddos_8,tbi_ddos_9,tbi_ddos_10,tbi_ddos_11,obese_ddos_0,obese_ddos_1,obese_ddos_2,obese_ddos_3,obese_ddos_4,obese_ddos_5,obese_ddos_6,obese_ddos_7,obese_ddos_8,obese_ddos_9,obese_ddos_10,obese_ddos_11,pressure_ulcer_ddos_0,pressure_ulcer_ddos_1,pressure_ulcer_ddos_2,pressure_ulcer_ddos_3,pressure_ulcer_ddos_4,pressure_ulcer_ddos_5,pressure_ulcer_ddos_6,pressur

In [61]:
x_train, y_train = cb_model_utils.get_xy(train)

In [62]:
x_train.head()

,oxygen_ddos_0,oxygen_ddos_1,oxygen_ddos_2,oxygen_ddos_3,oxygen_ddos_4,oxygen_ddos_5,oxygen_ddos_6,oxygen_ddos_7,oxygen_ddos_8,oxygen_ddos_9,oxygen_ddos_10,oxygen_ddos_11,hosp_bed_ddos_0,hosp_bed_ddos_1,hosp_bed_ddos_2,hosp_bed_ddos_3,hosp_bed_ddos_4,hosp_bed_ddos_5,hosp_bed_ddos_6,hosp_bed_ddos_7,hosp_bed_ddos_8,hosp_bed_ddos_9,hosp_bed_ddos_10,hosp_bed_ddos_11,chf_ddos_0,chf_ddos_1,chf_ddos_2,chf_ddos_3,chf_ddos_4,chf_ddos_5,chf_ddos_6,chf_ddos_7,chf_ddos_8,chf_ddos_9,chf_ddos_10,chf_ddos_11,heart_ddos_0,heart_ddos_1,heart_ddos_2,heart_ddos_3,heart_ddos_4,heart_ddos_5,heart_ddos_6,heart_ddos_7,heart_ddos_8,heart_ddos_9,heart_ddos_10,heart_ddos_11,copd_ddos_0,copd_ddos_1,copd_ddos_2,copd_ddos_3,copd_ddos_4,copd_ddos_5,copd_ddos_6,copd_ddos_7,copd_ddos_8,copd_ddos_9,copd_ddos_10,copd_ddos_11,pulmonar_ddos_0,pulmonar_ddos_1,pulmonar_ddos_2,pulmonar_ddos_3,pulmonar_ddos_4,pulmonar_ddos_5,pulmonar_ddos_6,pulmonar_ddos_7,pulmonar_ddos_8,pulmonar_ddos_9,pulmonar_ddos_10,pulmonar_ddos_11,cancer_ddos_0,cancer_ddos_1,cancer_ddos_2,cancer_ddos_3,cancer_ddos_4,cancer_ddos_5,cancer_ddos_6,cancer_ddos_7,cancer_ddos_8,cancer_ddos_9,cancer_ddos_10,cancer_ddos_11,ckd_ddos_0,ckd_ddos_1,ckd_ddos_2,ckd_ddos_3,ckd_ddos_4,ckd_ddos_5,ckd_ddos_6,ckd_ddos_7,ckd_ddos_8,ckd_ddos_9,ckd_ddos_10,ckd_ddos_11,esrd_ddos_0,esrd_ddos_1,esrd_ddos_2,esrd_ddos_3,esrd_ddos_4,esrd_ddos_5,esrd_ddos_6,esrd_ddos_7,esrd_ddos_8,esrd_ddos_9,esrd_ddos_10,esrd_ddos_11,lipidy_ddos_0,lipidy_ddos_1,lipidy_ddos_2,lipidy_ddos_3,lipidy_ddos_4,lipidy_ddos_5,lipidy_ddos_6,lipidy_ddos_7,lipidy_ddos_8,lipidy_ddos_9,lipidy_ddos_10,lipidy_ddos_11,diab_ddos_0,diab_ddos_1,diab_ddos_2,diab_ddos_3,diab_ddos_4,diab_ddos_5,diab_ddos_6,diab_ddos_7,diab_ddos_8,diab_ddos_9,diab_ddos_10,diab_ddos_11,alzh_ddos_0,alzh_ddos_1,alzh_ddos_2,alzh_ddos_3,alzh_ddos_4,alzh_ddos_5,alzh_ddos_6,alzh_ddos_7,alzh_ddos_8,alzh_ddos_9,alzh_ddos_10,alzh_ddos_11,demented_ddos_0,demented_ddos_1,demented_ddos_2,demented_ddos_3,demented_ddos_4,demented_ddos_5,demented_ddos_6,demented_ddos_7,demented_ddos_8,demented_ddos_9,demented_ddos_10,demented_ddos_11,stroke_ddos_0,stroke_ddos_1,stroke_ddos_2,stroke_ddos_3,stroke_ddos_4,stroke_ddos_5,stroke_ddos_6,stroke_ddos_7,stroke_ddos_8,stroke_ddos_9,stroke_ddos_10,stroke_ddos_11,hyper_ddos_0,hyper_ddos_1,hyper_ddos_2,hyper_ddos_3,hyper_ddos_4,hyper_ddos_5,hyper_ddos_6,hyper_ddos_7,hyper_ddos_8,hyper_ddos_9,hyper_ddos_10,hyper_ddos_11,fall_ddos_0,fall_ddos_1,fall_ddos_2,fall_ddos_3,fall_ddos_4,fall_ddos_5,fall_ddos_6,fall_ddos_7,fall_ddos_8,fall_ddos_9,fall_ddos_10,fall_ddos_11,trans_ddos_0,trans_ddos_1,trans_ddos_2,trans_ddos_3,trans_ddos_4,trans_ddos_5,trans_ddos_6,trans_ddos_7,trans_ddos_8,trans_ddos_9,trans_ddos_10,trans_ddos_11,liver_ddos_0,liver_ddos_1,liver_ddos_2,liver_ddos_3,liver_ddos_4,liver_ddos_5,liver_ddos_6,liver_ddos_7,liver_ddos_8,liver_ddos_9,liver_ddos_10,liver_ddos_11,hippy_ddos_0,hippy_ddos_1,hippy_ddos_2,hippy_ddos_3,hippy_ddos_4,hippy_ddos_5,hippy_ddos_6,hippy_ddos_7,hippy_ddos_8,hippy_ddos_9,hippy_ddos_10,hippy_ddos_11,depressed_ddos_0,depressed_ddos_1,depressed_ddos_2,depressed_ddos_3,depressed_ddos_4,depressed_ddos_5,depressed_ddos_6,depressed_ddos_7,depressed_ddos_8,depressed_ddos_9,depressed_ddos_10,depressed_ddos_11,psycho_ddos_0,psycho_ddos_1,psycho_ddos_2,psycho_ddos_3,psycho_ddos_4,psycho_ddos_5,psycho_ddos_6,psycho_ddos_7,psycho_ddos_8,psycho_ddos_9,...,para_ddos_10,para_ddos_11,quad_ddos_0,quad_ddos_1,quad_ddos_2,quad_ddos_3,quad_ddos_4,quad_ddos_5,quad_ddos_6,quad_ddos_7,quad_ddos_8,quad_ddos_9,quad_ddos_10,quad_ddos_11,tbi_ddos_0,tbi_ddos_1,tbi_ddos_2,tbi_ddos_3,tbi_ddos_4,tbi_ddos_5,tbi_ddos_6,tbi_ddos_7,tbi_ddos_8,tbi_ddos_9,tbi_ddos_10,tbi_ddos_11,obese_ddos_0,obese_ddos_1,obese_ddos_2,obese_ddos_3,obese_ddos_4,obese_ddos_5,obese_ddos_6,obese_ddos_7,obese_ddos_8,obese_ddos_9,obese_ddos_10,obese_ddos_11,pressure_ulcer_ddos_0,pressure_ulcer_ddos_1,pressure_ulcer_ddos_2,pressure_ulcer_ddos_3,pressure_ulcer_ddos_4,pressure_ulcer_ddos_5,pre

In [63]:
histr = HistGradientBoostingRegressor()
histr.fit(x_train, y_train)

HistGradientBoostingRegressor()

In [64]:
perf = cb_model_utils.get_model_performance(histr, train, val)
perf

{'train_score': 0.2797807513425262,
 'val_score': 0.17033048510352933,
 'train_mae': 10225.457482954036,
 'val_mae': 10586.90156073925}

In [65]:

param_grid = {
    'l2_regularization': [.5, 1, 2],
    'learning_rate': [0.01, 0.1, .2, .5, .7, 1],
    'min_samples_leaf': [20, 50, 100, 200],
    'max_iter': [100, 200, 500],
    'max_depth': [3, 5, 10, 20, 50],
}


base_estimator = HistGradientBoostingRegressor()
sh = HalvingRandomSearchCV(base_estimator,
                         param_grid,
                         cv=5,
                         factor=2,
                         n_candidates='exhaust',
                         min_resources=5000,
                         scoring='neg_mean_squared_error',
                         n_jobs=-1
                        ).fit(x_train, y_train)
sh.best_params_

/Users/bp/miniforge3/envs/data-analytics/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'min_samples_leaf': 200,
 'max_iter': 200,
 'max_depth': 10,
 'learning_rate': 0.1,
 'l2_regularization': 1}

In [66]:
perf = cb_model_utils.get_model_performance(sh.best_estimator_, train, val)
perf

{'train_score': 0.26001750859104855,
 'val_score': 0.17461825181104618,
 'train_mae': 10174.993719751497,
 'val_mae': 10554.440783571994}

### MOM st DDOS + Dem

In [60]:
d = cb_model_utils.build_mom_stdxddos_dem(master_df)
# train_val_test_split(d, 'yearly_st_ddos_dem', return_wo_saving=True)
train, val, test =  cb_model_utils.train_val_test_split(d, return_wo_saving=True)

train.head()

,oxygen_ddos_0,oxygen_ddos_1,oxygen_ddos_2,oxygen_ddos_3,oxygen_ddos_4,oxygen_ddos_5,oxygen_ddos_6,oxygen_ddos_7,oxygen_ddos_8,oxygen_ddos_9,oxygen_ddos_10,oxygen_ddos_11,hosp_bed_ddos_0,hosp_bed_ddos_1,hosp_bed_ddos_2,hosp_bed_ddos_3,hosp_bed_ddos_4,hosp_bed_ddos_5,hosp_bed_ddos_6,hosp_bed_ddos_7,hosp_bed_ddos_8,hosp_bed_ddos_9,hosp_bed_ddos_10,hosp_bed_ddos_11,chf_ddos_0,chf_ddos_1,chf_ddos_2,chf_ddos_3,chf_ddos_4,chf_ddos_5,chf_ddos_6,chf_ddos_7,chf_ddos_8,chf_ddos_9,chf_ddos_10,chf_ddos_11,heart_ddos_0,heart_ddos_1,heart_ddos_2,heart_ddos_3,heart_ddos_4,heart_ddos_5,heart_ddos_6,heart_ddos_7,heart_ddos_8,heart_ddos_9,heart_ddos_10,heart_ddos_11,copd_ddos_0,copd_ddos_1,copd_ddos_2,copd_ddos_3,copd_ddos_4,copd_ddos_5,copd_ddos_6,copd_ddos_7,copd_ddos_8,copd_ddos_9,copd_ddos_10,copd_ddos_11,pulmonar_ddos_0,pulmonar_ddos_1,pulmonar_ddos_2,pulmonar_ddos_3,pulmonar_ddos_4,pulmonar_ddos_5,pulmonar_ddos_6,pulmonar_ddos_7,pulmonar_ddos_8,pulmonar_ddos_9,pulmonar_ddos_10,pulmonar_ddos_11,cancer_ddos_0,cancer_ddos_1,cancer_ddos_2,cancer_ddos_3,cancer_ddos_4,cancer_ddos_5,cancer_ddos_6,cancer_ddos_7,cancer_ddos_8,cancer_ddos_9,cancer_ddos_10,cancer_ddos_11,ckd_ddos_0,ckd_ddos_1,ckd_ddos_2,ckd_ddos_3,ckd_ddos_4,ckd_ddos_5,ckd_ddos_6,ckd_ddos_7,ckd_ddos_8,ckd_ddos_9,ckd_ddos_10,ckd_ddos_11,esrd_ddos_0,esrd_ddos_1,esrd_ddos_2,esrd_ddos_3,esrd_ddos_4,esrd_ddos_5,esrd_ddos_6,esrd_ddos_7,esrd_ddos_8,esrd_ddos_9,esrd_ddos_10,esrd_ddos_11,lipidy_ddos_0,lipidy_ddos_1,lipidy_ddos_2,lipidy_ddos_3,lipidy_ddos_4,lipidy_ddos_5,lipidy_ddos_6,lipidy_ddos_7,lipidy_ddos_8,lipidy_ddos_9,lipidy_ddos_10,lipidy_ddos_11,diab_ddos_0,diab_ddos_1,diab_ddos_2,diab_ddos_3,diab_ddos_4,diab_ddos_5,diab_ddos_6,diab_ddos_7,diab_ddos_8,diab_ddos_9,diab_ddos_10,diab_ddos_11,alzh_ddos_0,alzh_ddos_1,alzh_ddos_2,alzh_ddos_3,alzh_ddos_4,alzh_ddos_5,alzh_ddos_6,alzh_ddos_7,alzh_ddos_8,alzh_ddos_9,alzh_ddos_10,alzh_ddos_11,demented_ddos_0,demented_ddos_1,demented_ddos_2,demented_ddos_3,demented_ddos_4,demented_ddos_5,demented_ddos_6,demented_ddos_7,demented_ddos_8,demented_ddos_9,demented_ddos_10,demented_ddos_11,stroke_ddos_0,stroke_ddos_1,stroke_ddos_2,stroke_ddos_3,stroke_ddos_4,stroke_ddos_5,stroke_ddos_6,stroke_ddos_7,stroke_ddos_8,stroke_ddos_9,stroke_ddos_10,stroke_ddos_11,hyper_ddos_0,hyper_ddos_1,hyper_ddos_2,hyper_ddos_3,hyper_ddos_4,hyper_ddos_5,hyper_ddos_6,hyper_ddos_7,hyper_ddos_8,hyper_ddos_9,hyper_ddos_10,hyper_ddos_11,fall_ddos_0,fall_ddos_1,fall_ddos_2,fall_ddos_3,fall_ddos_4,fall_ddos_5,fall_ddos_6,fall_ddos_7,fall_ddos_8,fall_ddos_9,fall_ddos_10,fall_ddos_11,trans_ddos_0,trans_ddos_1,trans_ddos_2,trans_ddos_3,trans_ddos_4,trans_ddos_5,trans_ddos_6,trans_ddos_7,trans_ddos_8,trans_ddos_9,trans_ddos_10,trans_ddos_11,liver_ddos_0,liver_ddos_1,liver_ddos_2,liver_ddos_3,liver_ddos_4,liver_ddos_5,liver_ddos_6,liver_ddos_7,liver_ddos_8,liver_ddos_9,liver_ddos_10,liver_ddos_11,hippy_ddos_0,hippy_ddos_1,hippy_ddos_2,hippy_ddos_3,hippy_ddos_4,hippy_ddos_5,hippy_ddos_6,hippy_ddos_7,hippy_ddos_8,hippy_ddos_9,hippy_ddos_10,hippy_ddos_11,depressed_ddos_0,depressed_ddos_1,depressed_ddos_2,depressed_ddos_3,depressed_ddos_4,depressed_ddos_5,depressed_ddos_6,depressed_ddos_7,depressed_ddos_8,depressed_ddos_9,depressed_ddos_10,depressed_ddos_11,psycho_ddos_0,psycho_ddos_1,psycho_ddos_2,psycho_ddos_3,psycho_ddos_4,psycho_ddos_5,psycho_ddos_6,psycho_ddos_7,psycho_ddos_8,psycho_ddos_9,...,quad_ddos_0,quad_ddos_1,quad_ddos_2,quad_ddos_3,quad_ddos_4,quad_ddos_5,quad_ddos_6,quad_ddos_7,quad_ddos_8,quad_ddos_9,quad_ddos_10,quad_ddos_11,tbi_ddos_0,tbi_ddos_1,tbi_ddos_2,tbi_ddos_3,tbi_ddos_4,tbi_ddos_5,tbi_ddos_6,tbi_ddos_7,tbi_ddos_8,tbi_ddos_9,tbi_ddos_10,tbi_ddos_11,obese_ddos_0,obese_ddos_1,obese_ddos_2,obese_ddos_3,obese_ddos_4,obese_ddos_5,obese_ddos_6,obese_ddos_7,obese_ddos_8,obese_ddos_9,obese_ddos_10,obese_ddos_11,pressure_ulcer_ddos_0,pressure_ulcer_ddos_1,pressure_ulcer_ddos_2,pressure_ulcer_ddos_3,pressure_ulcer_ddos_4,pressure_ulcer_ddos_5,pressure_ulcer_ddos_6,pressur

In [61]:
x_train, y_train = cb_model_utils.get_xy(train)

In [62]:
x_train.head()

,oxygen_ddos_0,oxygen_ddos_1,oxygen_ddos_2,oxygen_ddos_3,oxygen_ddos_4,oxygen_ddos_5,oxygen_ddos_6,oxygen_ddos_7,oxygen_ddos_8,oxygen_ddos_9,oxygen_ddos_10,oxygen_ddos_11,hosp_bed_ddos_0,hosp_bed_ddos_1,hosp_bed_ddos_2,hosp_bed_ddos_3,hosp_bed_ddos_4,hosp_bed_ddos_5,hosp_bed_ddos_6,hosp_bed_ddos_7,hosp_bed_ddos_8,hosp_bed_ddos_9,hosp_bed_ddos_10,hosp_bed_ddos_11,chf_ddos_0,chf_ddos_1,chf_ddos_2,chf_ddos_3,chf_ddos_4,chf_ddos_5,chf_ddos_6,chf_ddos_7,chf_ddos_8,chf_ddos_9,chf_ddos_10,chf_ddos_11,heart_ddos_0,heart_ddos_1,heart_ddos_2,heart_ddos_3,heart_ddos_4,heart_ddos_5,heart_ddos_6,heart_ddos_7,heart_ddos_8,heart_ddos_9,heart_ddos_10,heart_ddos_11,copd_ddos_0,copd_ddos_1,copd_ddos_2,copd_ddos_3,copd_ddos_4,copd_ddos_5,copd_ddos_6,copd_ddos_7,copd_ddos_8,copd_ddos_9,copd_ddos_10,copd_ddos_11,pulmonar_ddos_0,pulmonar_ddos_1,pulmonar_ddos_2,pulmonar_ddos_3,pulmonar_ddos_4,pulmonar_ddos_5,pulmonar_ddos_6,pulmonar_ddos_7,pulmonar_ddos_8,pulmonar_ddos_9,pulmonar_ddos_10,pulmonar_ddos_11,cancer_ddos_0,cancer_ddos_1,cancer_ddos_2,cancer_ddos_3,cancer_ddos_4,cancer_ddos_5,cancer_ddos_6,cancer_ddos_7,cancer_ddos_8,cancer_ddos_9,cancer_ddos_10,cancer_ddos_11,ckd_ddos_0,ckd_ddos_1,ckd_ddos_2,ckd_ddos_3,ckd_ddos_4,ckd_ddos_5,ckd_ddos_6,ckd_ddos_7,ckd_ddos_8,ckd_ddos_9,ckd_ddos_10,ckd_ddos_11,esrd_ddos_0,esrd_ddos_1,esrd_ddos_2,esrd_ddos_3,esrd_ddos_4,esrd_ddos_5,esrd_ddos_6,esrd_ddos_7,esrd_ddos_8,esrd_ddos_9,esrd_ddos_10,esrd_ddos_11,lipidy_ddos_0,lipidy_ddos_1,lipidy_ddos_2,lipidy_ddos_3,lipidy_ddos_4,lipidy_ddos_5,lipidy_ddos_6,lipidy_ddos_7,lipidy_ddos_8,lipidy_ddos_9,lipidy_ddos_10,lipidy_ddos_11,diab_ddos_0,diab_ddos_1,diab_ddos_2,diab_ddos_3,diab_ddos_4,diab_ddos_5,diab_ddos_6,diab_ddos_7,diab_ddos_8,diab_ddos_9,diab_ddos_10,diab_ddos_11,alzh_ddos_0,alzh_ddos_1,alzh_ddos_2,alzh_ddos_3,alzh_ddos_4,alzh_ddos_5,alzh_ddos_6,alzh_ddos_7,alzh_ddos_8,alzh_ddos_9,alzh_ddos_10,alzh_ddos_11,demented_ddos_0,demented_ddos_1,demented_ddos_2,demented_ddos_3,demented_ddos_4,demented_ddos_5,demented_ddos_6,demented_ddos_7,demented_ddos_8,demented_ddos_9,demented_ddos_10,demented_ddos_11,stroke_ddos_0,stroke_ddos_1,stroke_ddos_2,stroke_ddos_3,stroke_ddos_4,stroke_ddos_5,stroke_ddos_6,stroke_ddos_7,stroke_ddos_8,stroke_ddos_9,stroke_ddos_10,stroke_ddos_11,hyper_ddos_0,hyper_ddos_1,hyper_ddos_2,hyper_ddos_3,hyper_ddos_4,hyper_ddos_5,hyper_ddos_6,hyper_ddos_7,hyper_ddos_8,hyper_ddos_9,hyper_ddos_10,hyper_ddos_11,fall_ddos_0,fall_ddos_1,fall_ddos_2,fall_ddos_3,fall_ddos_4,fall_ddos_5,fall_ddos_6,fall_ddos_7,fall_ddos_8,fall_ddos_9,fall_ddos_10,fall_ddos_11,trans_ddos_0,trans_ddos_1,trans_ddos_2,trans_ddos_3,trans_ddos_4,trans_ddos_5,trans_ddos_6,trans_ddos_7,trans_ddos_8,trans_ddos_9,trans_ddos_10,trans_ddos_11,liver_ddos_0,liver_ddos_1,liver_ddos_2,liver_ddos_3,liver_ddos_4,liver_ddos_5,liver_ddos_6,liver_ddos_7,liver_ddos_8,liver_ddos_9,liver_ddos_10,liver_ddos_11,hippy_ddos_0,hippy_ddos_1,hippy_ddos_2,hippy_ddos_3,hippy_ddos_4,hippy_ddos_5,hippy_ddos_6,hippy_ddos_7,hippy_ddos_8,hippy_ddos_9,hippy_ddos_10,hippy_ddos_11,depressed_ddos_0,depressed_ddos_1,depressed_ddos_2,depressed_ddos_3,depressed_ddos_4,depressed_ddos_5,depressed_ddos_6,depressed_ddos_7,depressed_ddos_8,depressed_ddos_9,depressed_ddos_10,depressed_ddos_11,psycho_ddos_0,psycho_ddos_1,psycho_ddos_2,psycho_ddos_3,psycho_ddos_4,psycho_ddos_5,psycho_ddos_6,psycho_ddos_7,psycho_ddos_8,psycho_ddos_9,...,para_ddos_10,para_ddos_11,quad_ddos_0,quad_ddos_1,quad_ddos_2,quad_ddos_3,quad_ddos_4,quad_ddos_5,quad_ddos_6,quad_ddos_7,quad_ddos_8,quad_ddos_9,quad_ddos_10,quad_ddos_11,tbi_ddos_0,tbi_ddos_1,tbi_ddos_2,tbi_ddos_3,tbi_ddos_4,tbi_ddos_5,tbi_ddos_6,tbi_ddos_7,tbi_ddos_8,tbi_ddos_9,tbi_ddos_10,tbi_ddos_11,obese_ddos_0,obese_ddos_1,obese_ddos_2,obese_ddos_3,obese_ddos_4,obese_ddos_5,obese_ddos_6,obese_ddos_7,obese_ddos_8,obese_ddos_9,obese_ddos_10,obese_ddos_11,pressure_ulcer_ddos_0,pressure_ulcer_ddos_1,pressure_ulcer_ddos_2,pressure_ulcer_ddos_3,pressure_ulcer_ddos_4,pressure_ulcer_ddos_5,pre

In [63]:
histr = HistGradientBoostingRegressor()
histr.fit(x_train, y_train)

HistGradientBoostingRegressor()

In [64]:
perf = cb_model_utils.get_model_performance(histr, train, val)
perf

{'train_score': 0.2797807513425262,
 'val_score': 0.17033048510352933,
 'train_mae': 10225.457482954036,
 'val_mae': 10586.90156073925}

In [65]:

param_grid = {
    'l2_regularization': [.5, 1, 2],
    'learning_rate': [0.01, 0.1, .2, .5, .7, 1],
    'min_samples_leaf': [20, 50, 100, 200],
    'max_iter': [100, 200, 500],
    'max_depth': [3, 5, 10, 20, 50],
}


base_estimator = HistGradientBoostingRegressor()
sh = HalvingRandomSearchCV(base_estimator,
                         param_grid,
                         cv=5,
                         factor=2,
                         n_candidates='exhaust',
                         min_resources=5000,
                         scoring='neg_mean_squared_error',
                         n_jobs=-1
                        ).fit(x_train, y_train)
sh.best_params_

/Users/bp/miniforge3/envs/data-analytics/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


{'min_samples_leaf': 200,
 'max_iter': 200,
 'max_depth': 10,
 'learning_rate': 0.1,
 'l2_regularization': 1}

In [66]:
perf = cb_model_utils.get_model_performance(sh.best_estimator_, train, val)
perf

{'train_score': 0.26001750859104855,
 'val_score': 0.17461825181104618,
 'train_mae': 10174.993719751497,
 'val_mae': 10554.440783571994}

# Build and deploy
### Save data

In [ ]:
d = cb_model_utils.build_mom_stddos_dem(master_df)
cols = ['target'] + [c for c in d.columns if c != 'target' and c != 'member_id']  # sm target always first
d[cols].to_csv(f'data/mom_stddos_dem_12m_target.csv', header=False, index=False)

### deploy

## OLD

In [ ]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, val_x, val_y, n_repeats=10, random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{x.columns[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

In [ ]:
MOM DDOS service_types and dx + Dem
5...8 same but for TC

Maybe a combo TC + DDOS
Span?

### Normalize/encode features if needed
not needed for trees, most linear models will do it for you if you pass the param

In [ ]:
def mae(model, x, y):
    preds = model.predict(x)
    return np.abs(preds-y).mean()

### Tune mom ddos st + dem

In [ ]:
d = build_mom_stddos_dem(master_df)
# d = build_mom_sttc_dem(master_df)
# d = build_yearly_sttc_dem(master_df)
# d = build_yearly_stddos_dem(master_df)
train, val, test =  train_val_test_split(d, return_wo_saving=True)
train.head()

In [ ]:
x_cols = [c for c in train.columns if c not in ['member_id', 'target', 'period']]
# x_cols = [c for c in training_df.columns if c not in ['member_id', 'target', 'period'] + cat_feats]
x = train[x_cols]
y = train.target

val_x = val[x_cols]
val_y = val.target

Things to tune
- l2_regularization=1
- learning_rate
- min_samples_leaf
- max_iter=100,
- max_depth=40,

- tune('learning_rate', np.arange(0.01, 0.1, .01))
- min_samples_leaf_results = tune('min_samples_leaf', [50, 60, 70, 100])

In [ ]:
# determine number of iterations

In [ ]:
def tune(param, values):
    val_scores = []
    train_scores = []
    val_mae = []
    train_mae = []
    for v in tqdm(values):
        kw = {param: v}
        histr = HistGradientBoostingRegressor(max_iter=100, **kw)
        histr.fit(x, y)
        val_scores.append(histr.score(val_x, val_y))
        train_scores.append(histr.score(x, y))
        train_mae.append(np.abs(histr.predict(x) - y).mean())
        val_mae.append(np.abs(histr.predict(val_x) - val_y).mean())
        
#         print(f'Val: {val_scores[-1]}. Train: {train_scores[-1]}')

    results = pd.DataFrame(zip(values, train_scores, val_scores, train_mae, val_mae), columns=[param, 'Train Score', 'Val Score', 'Train MAE', 'Val MAE'])
    scores_tall = results.melt(id_vars=[param], value_vars=['Train Score', 'Val Score'], value_name='score')
    mae_tall = results.melt(id_vars=[param], value_vars=['Train MAE', 'Val MAE'], value_name='mae')

    display(sns.relplot(data=scores_tall, x=param, y='score', hue='variable', kind='line'))
    display(sns.relplot(data=mae_tall, x=param, y='mae', hue='variable', kind='line'))
    return results

In [ ]:
max_depth = tune('max_depth', [3, 5, 10, 20, 40, 75])

In [ ]:
l2 = tune('l2_regularization', np.arange(1., 3, .25))

In [ ]:
min_samples_leaf_results = tune('min_samples_leaf', [25, 50, 75, 100, 125])

In [ ]:
model = HistGradientBoostingRegressor(max_iter=100,
                                      min_samples_leaf=100,
                                      max_depth=5,
                                      l2_regularization=1.5)
model.fit(x,y)
model.score(x,y), model.score(val_x, val_y), np.abs(model.predict(x) - y).mean(), np.abs(model.predict(val_x) - val_y).mean()

In [ ]:
# mom tc
model.fit(x,y)
model.score(x,y), model.score(val_x, val_y)

In [ ]:
# mom tc
model.fit(x,y)
model.score(x,y), model.score(val_x, val_y)

In [ ]:
# mom ddos
model.fit(x,y)
model.score(x,y), model.score(val_x, val_y)

In [ ]:
# yearly tc
model.fit(x,y)
model.score(x,y), model.score(val_x, val_y)

In [ ]:
# yearly ddos
model.fit(x,y)
model.score(x,y), model.score(val_x, val_y)

In [ ]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, val_x, val_y, n_repeats=10, random_state=0)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{x.columns[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

In [ ]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{x.columns[i]:<8}"
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")

In [ ]:
ridge = Ridge(alpha=1, normalize=True)

In [ ]:
ridge.fit(x, y)

In [ ]:
ridge.score(x,y)

In [ ]:
val_x = val_df[x_cols]
val_y = val_df.target
ridge.score(val_x,val_y)

In [ ]:
histr = HistGradientBoostingRegressor()
histr.fit(x, y)

In [ ]:
histr.score(x, y)

In [ ]:
histr.score(val_x, val_y)

In [ ]:
val_df.head()

In [ ]:
train_preds = histr.predict(x)
val_preds = histr.predict(val_x)

In [ ]:
train_w_preds = training_df.assign(pred=train_preds, sample='train').merge(periods_df, on='period')
val_w_preds = val_df.assign(pred=val_preds, sample='validation').merge(periods_df, on='period')

In [ ]:
out_cols = ['member_id', 'sample', 'target', 'pred', 'period', 'pre_start', 'pre_end', 'post_start', 'post_end']

In [ ]:
pd.concat([train_w_preds[out_cols], val_w_preds[out_cols]]).to_csv('hgbr_12_mom_ddos.csv', index=False)

### Feature importance

In [ ]:
sorted(zip(ridge.coef_, x.columns))

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
result = permutation_importance(histr, val_x, val_y, n_repeats=10,random_state=0, n_jobs=-1)

In [ ]:
sorted(zip(result.importances_mean, val_x.columns))

### Write datasets for sagemaker

In [ ]:
export_cols = ['target'] + [c for c in training_df.columns if c not in ['member_id', 'target', 'period']]
training_df[export_cols].head()

In [ ]:
training_df[export_cols].to_csv('./data/train_df.csv', index=False, header=False)

In [ ]:
val_df[export_cols].to_csv('./data/val_df.csv', index=False, header=False)

In [ ]:
test_df[export_cols].to_csv('./data/test_df.csv', index=False, header=False)